In [1]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

# Move to the parent directory
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)

# Print the updated working directory
updated_directory = os.getcwd()
print("Updated Directory:", updated_directory)

Current Directory: /mnt/c/users/rjmay/code/research_ms/alt_distr
Updated Directory: /mnt/c/users/rjmay/code/research_ms


In [2]:
! source ../venv/bin/activate

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import math
import pickle
import time



In [4]:
PICKLED =True

dim_str = "15" # TODO change me 


In [5]:
BER_values = {'beta_beta': 0.10937546875016925, 'paired_uniforms': .1, 'normal_beta': 0.0726617598018901,
               'uniform_normal': 0.15350691875939732}

In [6]:
for k in BER_values:
    print((k))

beta_beta
paired_uniforms
normal_beta
uniform_normal


In [7]:
wanted_columns = ['Dp', 'Bha',
       'Bha_knn', 'inf', 'enDive']

### Simulation stage

In [8]:
from modules.multi_bounds_v3 import bounds_class
from modules.Bhatt_knn_func import knn_num_calc
from modules.data_gen import data_gen

In [9]:
file_path = 'sim_data/beta_beta' + dim_str +'.pkl' # DONT FORGET TO CHANGE ME IF YOU COPY AND PASTE


    

if PICKLED == True:
    

    # Open the file in binary read mode
    with open(file_path, 'rb') as file:
        # Use pickle.load to deserialize and load the list of objects from the file
        loaded_objects = pickle.load(file)

    print('Loaded object successfully')
    
    bound_obj_lst = loaded_objects

Loaded object successfully


In [10]:
all_obj = {}

for k in BER_values:
    file_path = 'sim_data/' + k + dim_str + '.pkl'
    print(file_path, end = '\t')
    #Open the file in binary read mode
    with open(file_path, 'rb') as file:
        # Use pickle.load to deserialize and load the list of objects from the file
        loaded_objects = pickle.load(file)

    print('Loaded object successfully')
    
    all_obj[k] = loaded_objects



sim_data/beta_beta15.pkl	Loaded object successfully
sim_data/paired_uniforms15.pkl	Loaded object successfully
sim_data/normal_beta15.pkl	Loaded object successfully
sim_data/uniform_normal15.pkl	Loaded object successfully


In [11]:
# all_obj

### Using the data



In [12]:
sample_sizes = np.logspace(2, 3.3011, 9 , endpoint = True, dtype = int)

dpi = 250

In [13]:
validity_dict = {}

for k in all_obj:
    temp_bound_obj_lst = all_obj[k]

    lst = [obj.validity(BER_values[k]) for obj in temp_bound_obj_lst ]
    df_valid = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = df_valid[wanted_columns]

    short_df = pd.concat([good_columns.head(1), good_columns.tail(1)])
    
    validity_dict[k] = short_df



In [14]:
validity_dict

{'beta_beta':        Dp     Bha  Bha_knn  inf  enDive
 100   0.0  0.9675   0.0975  0.0  0.0550
 2000  0.0  1.0000   0.0000  0.0  0.1875,
 'paired_uniforms':        Dp     Bha  Bha_knn  inf  enDive
 100   0.0  0.9975     0.66  0.0   0.045
 2000  0.0  1.0000     0.00  0.0   0.170,
 'normal_beta':        Dp   Bha  Bha_knn  inf  enDive
 100   0.0  0.98    0.005  0.0  0.0175
 2000  0.0  1.00    0.000  0.0  0.1125,
 'uniform_normal':           Dp    Bha  Bha_knn  inf  enDive
 100   0.3575  0.545      1.0  0.0  0.0650
 2000  0.9425  1.000      1.0  0.0  0.2325}

In [15]:
# final_df = pd.concat(
#     [df.assign(Key=key) for key, df in validity_dict.items()],
#     ignore_index=True
# ) 

# final_df

In [16]:
#  Assuming validity_dict is populated as per your previous code
final_df_list = []

for key, df in validity_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_validity = pd.concat(final_df_list, ignore_index=True)

# Display the final DataFrame
final_validity

,Dp,Bha,Bha_knn,inf,enDive,Key
0,0.0000,0.9675,0.0975,0.0,0.0550,beta_beta_100
1,0.0000,1.0000,0.0000,0.0,0.1875,beta_beta_2000
2,0.0000,0.9975,0.6600,0.0,0.0450,paired_uniforms_100
3,0.0000,1.0000,0.0000,0.0,0.1700,paired_uniforms_2000
4,0.0000,0.9800,0.0050,0.0,0.0175,normal_beta_100
5,0.0000,1.0000,0.0000,0.0,0.1125,normal_beta_2000
6,0.3575,0.5450,1.0000,0.0,0.0650,uniform_normal_100
7,0.9425,1.0000,1.0000,0.0,0.2325,uniform_normal_2000


In [17]:
lst = [obj.bound_width() for obj in bound_obj_lst ]

bound_width_df = pd.DataFrame(lst, index = sample_sizes)

bound_width_df.head()

,Dp,Bha,Bha_knn,tight,inf,enDive
100,0.105324,0.173924,0.204994,NaN,0.007139,0.021560
145,0.109448,0.168962,0.205149,NaN,0.008133,0.025009
211,0.115048,0.166714,0.205333,NaN,0.009910,0.027421
307,0.117838,0.165666,0.205334,NaN,0.012013,0.036091
447,0.119755,0.163955,0.205446,NaN,0.014399,0.042410


In [18]:
width_dict = {}

for k in all_obj:
    temp_bound_obj_lst = all_obj[k]

    lst = [obj.bound_width() for obj in temp_bound_obj_lst ]
    temp_df = pd.DataFrame(lst, index = sample_sizes)

    good_columns  = temp_df[wanted_columns]

    short_df = pd.concat([good_columns.head(1), good_columns.tail(1)])
    
    width_dict[k] = short_df

In [19]:
#  Assuming validity_dict is populated as per your previous code
final_df_list2 = []

for key, df in width_dict.items():
    # Create a new column combining the key and index
    df = df.copy()  # Avoid modifying the original DataFrame
    df['Key'] = [f'{key}_{idx}' for idx in df.index]
    
    # Append the DataFrame with the new Key column to the list
    final_df_list2.append(df)

# Concatenate the list of DataFrames into one final DataFrame
final_width = pd.concat(final_df_list2, ignore_index=True)

# Display the final DataFrame
final_width

,Dp,Bha,Bha_knn,inf,enDive,Key
0,0.105324,0.173924,0.204994,0.007139,0.021560,beta_beta_100
1,0.124257,0.161005,0.203497,0.004778,0.047099,beta_beta_2000
2,0.120880,0.168759,0.197878,0.007287,0.021956,paired_uniforms_100
3,0.109261,0.154279,0.205447,0.004643,0.036690,paired_uniforms_2000
4,0.121947,0.172871,0.199174,0.006326,0.021374,normal_beta_100
5,0.120802,0.150710,0.205867,0.005505,0.076327,normal_beta_2000
6,0.108598,0.203119,0.176454,0.005675,0.023114,uniform_normal_100
7,0.102454,0.199121,0.189334,0.005787,0.050659,uniform_normal_2000


In [20]:
final_validity_renamed = final_validity.rename(columns={
    'Dp': 'Dp_valid',
    'Bha': 'Bha_valid',
    'Bha_knn': 'Bha_knn_valid',
    'inf': 'inf_valid',
    'enDive': 'enDive_valid'
})

final_width_renamed = final_width.rename(columns={
    'Dp': 'Dp_width',
    'Bha': 'Bha_width',
    'Bha_knn': 'Bha_knn_width',
    'inf': 'inf_width',
    'enDive': 'enDive_width'
})

# Step 2: Merge the two DataFrames on the 'Key' column
combined_df = pd.merge(final_validity_renamed, final_width_renamed, on='Key')

# Display the combined DataFrame
columns = combined_df.columns.to_list()

columns.sort()
# combined_df.columns
columns

['Bha_knn_valid',
 'Bha_knn_width',
 'Bha_valid',
 'Bha_width',
 'Dp_valid',
 'Dp_width',
 'Key',
 'enDive_valid',
 'enDive_width',
 'inf_valid',
 'inf_width']

In [21]:
order_columns = [ 'Key',
'Bha_valid',
'Bha_width',
'Bha_knn_valid',
'Bha_knn_width',
'inf_valid',
'inf_width',
 'Dp_valid',
 'Dp_width',
 'enDive_valid',
 'enDive_width'
 ]

In [22]:
combined_sorted = combined_df[order_columns]

combined_sorted.index = combined_sorted.Key

combined_sorted= combined_sorted.drop(columns=['Key'])

combined_sorted

,Bha_valid,Bha_width,Bha_knn_valid,Bha_knn_width,inf_valid,inf_width,Dp_valid,Dp_width,enDive_valid,enDive_width
Key,,,,,,,,,,
beta_beta_100,0.9675,0.173924,0.0975,0.204994,0.0,0.007139,0.0000,0.105324,0.0550,0.021560
beta_beta_2000,1.0000,0.161005,0.0000,0.203497,0.0,0.004778,0.0000,0.124257,0.1875,0.047099
paired_uniforms_100,0.9975,0.168759,0.6600,0.197878,0.0,0.007287,0.0000,0.120880,0.0450,0.021956
paired_uniforms_2000,1.0000,0.154279,0.0000,0.205447,0.0,0.004643,0.0000,0.109261,0.1700,0.036690
normal_beta_100,0.9800,0.172871,0.0050,0.199174,0.0,0.006326,0.0000,0.121947,0.0175,0.021374
normal_beta_2000,1.0000,0.150710,0.0000,0.205867,0.0,0.005505,0.0000,0.120802,0.1125,0.076327
uniform_normal_100,0.5450,0.203119,1.0000,0.176454,0.0,0.005675,0.3575,0.108598,0.0650,0.023114
uniform_normal_2000,1.0000,0.199121,1.0000,0.189334,0.0,0.005787,0.9425,0.102454,0.2325,0.050659
